In [1]:
import sys, os
p = os.path.abspath('..')
# p = p+r'\config'
sys.path.insert(1, p)

In [13]:
import psycopg2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import time
import pprint
from itertools import combinations
import pickle
import config.config as conf
import re

In [3]:
conn = psycopg2.connect(host = conf.database_user['host'], dbname=conf.database_user['dbname'], user=conf.database_user['user'], password=conf.database_user['password'])
try:
    cur = conn.cursor()
    cur.execute("\
select\
                      x.q_id\
                    , x.q_posttypeid\
                    , x.q_acceptedanswerid\
                    , x.q_parentid\
                    , x.q_creationdate\
                    , x.q_score\
                    , x.q_viewcount\
                    , x.q_owneruserid\
                    , x.q_title\
                    , x.q_tags\
                    , x.q_answercount\
                    , x.q_commentcount\
                    , x.q_reputation\
                    , f.body             as q_body\
                    , b.id               as a_id\
                    , b.posttypeid       as a_posttypeid\
                    , b.acceptedanswerid as a_acceptedanswerid\
                    , b.parentid         as a_parentid\
                    , b.creationdate     as a_creationdate\
                    , b.score            as a_score\
                    , b.viewcount        as a_viewcount\
                    , b.owneruserid      as a_owneruserid\
                    , b.title            as a_title\
                    , b.tags             as a_tags\
                    , b.answercount      as a_answercount\
                    , b.commentcount     as a_commentcount\
                    , d.reputation       as a_reputation\
                    , g.body             as a_body\
      from (\
               select a.id               as q_id\
                    , a.posttypeid       as q_posttypeid\
                    , a.acceptedanswerid as q_acceptedanswerid\
                    , a.parentid         as q_parentid\
                    , a.creationdate     as q_creationdate\
                    , a.score            as q_score\
                    , a.viewcount        as q_viewcount\
                    , a.owneruserid      as q_owneruserid\
                    , a.title            as q_title\
                    , a.answercount      as q_answercount\
                    , a.commentcount     as q_commentcount\
                    , c.reputation       as q_reputation\
                    , replace(replace(lower(a.tags), '<', ''), '>', ' ')as q_tags\
               from posts a\
                  , users c\
               where a.creationdate >= '2023-11-17'\
                 and a.posttypeid = '1'\
                 and a.owneruserid is not null\
                 and c.id = a.owneruserid\
                 and (replace(replace(lower(a.tags), '<', ''), '>', ' ') like '%python %')\
           )   x\
            , posts b\
            , users d\
            , postsbody f\
            , postsbody g\
    where b.parentid = x.q_id\
      and b.posttypeid = '2'\
      and b.owneruserid is not null\
      and d.id = b.owneruserid\
      and x.q_id = f.id \
      and b.id = g.id \
; \
                " 
   )
    rows = cur.fetchall()
    

except psycopg2.DatabaseError as db_err:
    print(db_err)
finally : 
  cur.close()

In [4]:
df = pd.DataFrame(rows, columns = [
  'q_id' 
, 'q_posttypeid' 
, 'q_acceptedanswerid'
, 'q_parentid' 
, 'q_creationdate' 
, 'q_score' 
, 'q_viewcount' 
, 'q_owneruserid' 
, 'q_title' 
, 'q_tags' 
, 'q_answercount' 
, 'q_commentcount' 
, 'q_reputation' 
, 'q_text' 
, 'a_id' 
, 'a_posttypeid' 
, 'a_acceptedanswerid' 
, 'a_parentid' 
, 'a_creationdate' 
, 'a_score' 
, 'a_viewcount' 
, 'a_owneruserid' 
, 'a_title' 
, 'a_tags' 
, 'a_answercount'
, 'a_commentcount'
, 'a_reputation' 
, 'a_text' 
])

In [5]:
df.head()

,q_id,q_posttypeid,q_acceptedanswerid,q_parentid,q_creationdate,q_score,q_viewcount,q_owneruserid,q_title,q_tags,...,a_creationdate,a_score,a_viewcount,a_owneruserid,a_title,a_tags,a_answercount,a_commentcount,a_reputation,a_text
0,77593805,1,NaN,None,2023-12-03 10:05:29.263,-1,19,23031413,how to add number from 1 t0 20000 in python pr...,python,...,2023-12-03 10:11:55.577,0,None,14923149,None,None,None,0,381,<p>You can use the formula for the sum of an a...
1,77593717,1,NaN,None,2023-12-03 09:35:22.270,0,28,23031346,my python dehashing algoritm doesn't work,python hash,...,2023-12-03 10:11:57.450,0,None,17580381,None,None,None,0,21143,<p>A different structure to your code may help...
2,77591118,1,77591163.0,None,2023-12-02 16:07:09.687,1,23,5449454,"Pandas Series: split, modify first, join",python pandas series,...,2023-12-02 16:18:48.427,2,None,3185459,None,None,None,1,91367,<p>Use a single regex substitution instead:</p...
3,77590853,1,77591168.0,None,2023-12-02 14:54:18.170,-2,37,21429315,Python3 Nmap-APİ,python nmap,...,2023-12-02 16:19:08.763,0,None,19998536,None,None,None,0,12,<p>Make sure to install the python-nmap librar...
4,77591142,1,NaN,None,2023-12-02 16:14:58.957,-4,69,22834840,How to check whether a string is Palindrome or...,python,...,2023-12-02 16:19:24.450,-1,None,15261315,None,None,None,1,29096,"<p>No code because no attempt has been shown, ..."


In [6]:
print(df['q_creationdate'].min())
print(df['a_creationdate'].max())

2023-11-17 00:01:50.710000
2023-12-03 10:11:57.450000


In [7]:
# 전체 데이터셋 
with open('../../data/bert_df.pkl', 'wb') as f:
    pickle.dump(df, f)

In [8]:
# 파일 불러오기
with open('../../data/bert_df.pkl', 'rb') as f:
    df = pickle.load(f)


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3487 entries, 0 to 3486
Data columns (total 28 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   q_id                3487 non-null   int64         
 1   q_posttypeid        3487 non-null   object        
 2   q_acceptedanswerid  1488 non-null   float64       
 3   q_parentid          0 non-null      object        
 4   q_creationdate      3487 non-null   datetime64[ns]
 5   q_score             3487 non-null   int64         
 6   q_viewcount         3487 non-null   int64         
 7   q_owneruserid       3487 non-null   int64         
 8   q_title             3487 non-null   object        
 9   q_tags              3487 non-null   object        
 10  q_answercount       3487 non-null   int64         
 11  q_commentcount      3487 non-null   int64         
 12  q_reputation        3487 non-null   int64         
 13  q_text              3487 non-null   object      

In [26]:
# extract the  columns for text analysis
## q_body : question body
## a_body : answer body
df_q = df[['q_id','q_text']].drop_duplicates()
df_a = df[['a_id','a_text']].drop_duplicates()

In [27]:
def cleanhtml(raw_html):
  # 1.Source code in python language is hard to understand, so replace all the <code> tag first
  cleantext_1 = re.findall(r'(?<=\<code>)(.*?)(?=<\/code>)', raw_html.replace('\n', '_**_'))
  cleantext_1 = [x.replace('_**_', '\n') for x in cleantext_1]
  # 2. replace html tags
  # <p>
  tag_re = re.compile('<.*?>')
  cleantext_2 = [re.sub(tag_re, '', x) for x in cleantext_1]
  return cleantext_2

In [28]:
# apply the function, cleanhtml to the question and body text
df_q.loc[:, 'q_prep_text'] = df_q['q_text'].apply(cleanhtml)
df_a.loc[:, 'a_prep_text'] = df_a['a_text'].apply(cleanhtml)

In [33]:
df_q = df_q.reset_index(drop=True)
df_a = df_a.reset_index(drop=True)

In [35]:
df_q

,q_id,q_text,q_prep_text
0,77593805,<p>If we add number from 1 to 6 we have to ad...,[]
1,77593717,<pre><code>import hashlib\n\nuser_hash_dict = ...,[import hashlib\n\nuser_hash_dict = {}\n\nwith...
2,77591118,<p>I have a Pandas Series with the names of ci...,[London:Alpha\nLondon\nLondon:Beta\nLondon:Del...
3,77590853,"<p>Nmap API is written in Python and <code>-v,...","[-v, -o, -A, --script, -Pn, -IL]"
4,77591142,<p>I want to take a input of a string and want...,[]
...,...,...,...
2576,77575452,<p>I have many images from a high-speed camera...,[import cv2\nimport numpy as np\nfrom matplotl...
2577,77581957,"<p>I'm probably making a stupid mistake, but h...",[import seaborn as sns\nfrom scipy.stats impor...
2578,77581848,<p>I am trying to use Pandas to transform a df...,"[data = {'Account': ['A', 'A', 'A', 'A', 'A', ..."
2579,77567490,<p>I'm creating a local API for macOS that pro...,"[onnxruntime-silicon, onnxruntime, Flask, # Se..."


In [36]:
df_q_src = df_q[['q_id', 'q_prep_text']].apply(pd.Series.explode)
df_a_src = df_a[['a_id', 'a_prep_text']].apply(pd.Series.explode)

In [37]:
df_q_src['q_prep_text'] = df_q_src['q_prep_text'].str.split('\n')
df_a_src['a_prep_text'] = df_a_src['a_prep_text'].str.split('\n')

In [38]:
df_q_src.dropna(inplace=True)
df_a_src.dropna(inplace=True)
df_q = df_q.reset_index(drop=True)
df_a = df_a.reset_index(drop=True)

In [39]:
print(df_q_src['q_prep_text'].isna().sum())
print(df_a_src['a_prep_text'].isna().sum())

0
0


In [40]:
df_q_src['q_prep_text'] = df_q_src['q_prep_text'].apply(lambda x : [xx.lstrip().rstrip() for xx in x])
df_a_src['a_prep_text'] = df_a_src['a_prep_text'].apply(lambda x : [xx.lstrip().rstrip() for xx in x])

In [41]:
# 전체 데이터셋 
with open('../../data/q_src_df.pkl', 'wb') as f:
    pickle.dump(df_q_src, f)

with open('../../data/a_src_df.pkl', 'wb') as f:
    pickle.dump(df_a_src, f)